In [1]:
print(__doc__)

import numpy as np
import re
import json
import nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
    nltk.download("words")
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer

import csv

with open('emoji/emoji_final_join.json',encoding='utf-8') as f: 
    emoji_list = json.load(f)


stop_word = [line.rstrip('\r\n') for line in open("stop_words.txt")]

    

def split(word): 
    return list(word) 

def clean(text,emoji):
    am = text
    text = text.lower()
    
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()) 
    text = text.split()

    newtext = [x for x in text if not x.startswith('@') and not x.startswith('https')]
    newtext = [x for x in text if x not in stop_word]
    newtext = [WordNetLemmatizer().lemmatize(x,'v') for x in newtext]
    newtext = ' '.join(newtext)
    
    tokenizer = RegexpTokenizer(r'\w+')
    newtext = tokenizer.tokenize(newtext)
    #stop_words = stopwords.words('english')
    #newtext =  [w for w in newtext if not w in stop_words]
    #stopwords.words will delete word 'not' which is important for sentiment analysis, so we don't remove stopwords.
    stemmer = SnowballStemmer('english')
    newtext = list(map(stemmer.stem, newtext))
    emojis = split(emoji)
    newtext = newtext + emojis
    label = 0
    for ele in newtext:
        if ele =='amp':
            label = 1

    newtext = ' '.join(newtext)
    if label == 1:
        newtext = newtext.replace('amp','and')
   
    #print(newtext)
    return newtext
    



with open('data1.csv',encoding="utf8") as words_file:
    csv_reader = csv.DictReader(words_file, delimiter = ',')
    data = []
    for row in csv_reader:
        cleaned_row = []
        clean_text = clean(row['Text'],row['Emoji'])
        
        if len(clean_text) == 0:
            continue
            
        if float(row['Sentiment']) > 0 and int(row['Sentiment140']) == 4:
            sentiment = 'positive'
        elif float(row['Sentiment']) < 0 and int(row['Sentiment140']) == 0:
            sentiment = 'negative'
        elif float(row['Sentiment']) == 0 and int(row['Sentiment140']) == 2:
            if float(row['Emoji_sentiment']) > 0:
                sentiment = 'positive'
            elif float(row['Emoji_sentiment']) < 0:
                sentiment = 'negative'
            else:
                sentiment = 'neutral'
        elif float(row['Sentiment']) == 0:
            if int(row['Sentiment140']) == 0:
                sentiment = 'negative'
            if int(row['Sentiment140']) == 4:
                sentiment = 'positive'
        elif int(row['Sentiment140']) == 2:
            if float(row['Sentiment']) < 0:
                sentiment = 'negative'
            if float(row['Sentiment']) > 0:
                sentiment = 'positive'
        else:
            continue
            
        
        cleaned_row.append(clean_text)
        cleaned_row.append(sentiment)
        #cleaned_row.append(row['Emoji'])
        data.append(np.array(cleaned_row))
    data = np.array(data)
    
print(data[:20])
print(len(data))

Automatically created module for IPython interactive environment


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/iris0301/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


[['realiz tonight titti man 😂 😎' 'positive']
 ['capper' 'neutral']
 ['actin like realli want gon get fuck good get feel 😋' 'positive']
 ['rememb first hear song alway favorit' 'positive']
 ['don t need cupcak ken cmon' 'neutral']
 ['ikyfl 😭' 'negative']
 ['cant go back sleep' 'negative']
 ['haha true 😂' 'positive']
 ['bounc shit forev knee' 'negative']
 ['btw devour blog day love back' 'positive']
 ['get forev onli god know much love mrsgarza' 'positive']
 ['alway felt should lion senat presid glad neither' 'positive']
 ['daniel tell drink more wine while alreadi kinda buzz' 'positive']
 ['don t do drug kid 🤧' 'negative']
 ['twitter get troubl parent idc sad' 'negative']
 ['don t even get fade anymor' 'negative']
 ['win t help california forest fire' 'neutral']
 ['probabl would ve die go astro fest' 'negative']
 ['can disappear no one would notic' 'neutral']
 ['funclejustin' 'neutral']]
40870


In [50]:
Y = data[:,1]
X = data[:,0]
print(Y)
print(X)
print(len(X))

['positive' 'neutral' 'positive' ... 'positive' 'negative' 'negative']
['realiz tonight titti man 😂 😎' 'capper'
 'actin like realli want gon get fuck good get feel 😋' ...
 'wittelsbach magic' 'holi shit 🖤' 'see mean']
40870


In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern=r'[^\s]+')
newx = vectorizer.fit_transform(X)


In [61]:

feature_array = np.array(vectorizer.get_feature_names())
print(newx[0].toarray())
n = 5
X_new = []
for i in range(len(X)):
#     tfidf_sorting = np.argsort(newx[i].toarray())
#     tfidf_sorting=tfidf_sorting[0][::-1]
#     top_n = tfidf_sorting[:n]
#     X_new.append(list(top_n))
    X_new.append(newx[i].toarray()[0])
print(len(X_new))
#for col in newx.nonzero()[1]:
#    print(feature_array[col], ' - ', top_n[0, col])

[[0. 0. 0. ... 0. 0. 0.]]
40870


In [ ]:
# word_dict = dict()
# word_id = 0
# for ele in X:
#     ele = ele.split()
#     for word in ele:
#         if word not in word_dict:
#             word_dict[word] = word_id 
#             word_id += 1       
    
# X_id = []
    
# #print(word_dict)

In [ ]:

# all_dict = []

# for i in range(len(X)):
#     print(i)
#     wordDict = dict.fromkeys(word_dict, 0)
#     ele = np.array(X[i].split())
#     for word in ele:
#         wordDict[word] += 1
#     all_dict.append(wordDict)

In [ ]:
# Xnew = []
# for i in range(len(X)):
#     ele = X[i].split()
#     Xnew.append(ele)
# print(Xnew[:10])

In [ ]:
# import math
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD

# def computeTF(wordDict, bow):
#     tfDict = {}
#     bowCount = len(bow)
#     for word, count in wordDict.items():
#         tfDict[word] = count/float(bowCount)
#     return tfDict

# def computeIDF(docList):
#     idfDict = {}
#     N = len(docList)
    
#     idfDict = dict.fromkeys(docList[0].keys(), 0)
#     for doc in docList:
#         for word, val in doc.items():
#             if val > 0:
#                 idfDict[word] += 1
    
#     for word, val in idfDict.items():
#         idfDict[word] = math.log10(N / float(val))
        
#     return idfDict

# def computeTFIDF(tfBow, idfs):
#     tfidf = {}
#     for word, val in tfBow.items():
#         tfidf[word] = val*idfs[word]
#     return tfidf



# idfs = computeIDF(all_dict)
# result = []
# for i in range(len(all_dict)):
#     tf = computeTF(all_dict[i], Xnew[i])
#     tfidf = computeTFIDF(tf, idfs)
#     output = list(tfidf.values())
#     result.append(output)


# document_topic_matrix = TruncatedSVD(n_components=100, random_state=0).fit_transform(result)

# id_topic={}
# a = np.argsort(document_topic_matrix, axis=1)
# for i in range(len(document_topic_matrix)):
#     top5 = np.argsort(document_topic_matrix[i])[-5:][::-1]
#     id_topic[i] = list(top5)


# X_new = list(id_topic.values())

In [5]:
print(X_new[:10])

[[30220, 24934, 34244], [5572, 34441, 11479], [1444, 12901, 34241], [25224, 27922, 11047], [7798, 6645, 16612], [14944, 34275, 34441], [5532, 27548, 3203], [13545, 30735, 34232], [4598, 16880, 11707], [8703, 4279, 5026]]


In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.1, shuffle = True)


In [63]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
decision_tree = DecisionTreeClassifier(random_state=0)
decision_tree.fit(X_train, y_train)
decision_tree_score = decision_tree.score(X_test, y_test)
print('Decision Tree:', decision_tree_score)

KeyboardInterrupt: 

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=5,random_state=0)
clf.fit(X_train, y_train)  
print(clf.feature_importances_)
score = clf.score(X_test, y_test)
print('RF:', score)

[0.2625791  0.35935049 0.37807041]
RF: 0.4930266699290433


In [21]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)
knn_score = knn.score(X_test, y_test)
print('KNN:',knn_score)

KNN: 0.43919745534621973


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
transformed = tsne.fit_transform(X_new[:10000])
yy = []
for x in Y[:10000]:
    if x =='positive':
        yy.append('blue')
    if x=='negative':
        yy.append('red')
    if x =='neutral':
        yy.append('yellow')
print(len(yy))
colors = yy
plt.scatter(transformed[:, 0], transformed[:, 1],c=colors)
plt.colorbar()
plt.show()

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
svm = SVC(random_state=0)
svm.fit(X_train, y_train)
svm_score = svm.score(X_test, y_test)
print('SVM:',svm_score)

/Users/iris0301/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from gensim.models.phrases import Phrases, Phraser

tokenized_train = [t.split() for t in X]
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)
tg_phrases = Phrases(bigram[tokenized_train])
trigram = Phraser(tg_phrases)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle = True)

In [ ]:
import pandas as pd
from gensim.models.doc2vec import LabeledSentence, Doc2Vec, TaggedDocument

def labelize_tweets_bg(tweets,label):
    result = []
    prefix = label
    for i, t in enumerate(tweets):
        result.append(LabeledSentence(trigram[bigram[t.split()]], [prefix + '_%s' % i]))
    return result
  
#all_x = pd.concat([X_train,X_test])
all_x_w2v_bg = labelize_tweets_bg(X, 'all')


In [ ]:
from sklearn import utils
from tqdm import tqdm

#cores = multiprocessing.cpu_count()
model_bg_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, alpha=0.065, min_alpha=0.065)
model_bg_dbow.build_vocab([x for x in tqdm(all_x_w2v_bg)])

for epoch in range(30):
    model_bg_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dbow.alpha -= 0.002
    model_bg_dbow.min_alpha = model_bg_dbow.alpha
    

In [ ]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in range(len(corpus)):
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

train_vecs_dbow_bg = get_vectors(model_bg_dbow, X_train, 100)
validation_vecs_dbow_bg = get_vectors(model_bg_dbow, X_test, 100)


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_vecs_dbow_bg, y_train)
clf.score(validation_vecs_dbow_bg, y_test)